In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dropout, LSTM, Dense, Embedding

In [2]:
data = pd.read_csv('../data/preprocessed_data.csv')
text = data['text']

In [3]:
# text = text.sum()

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
encoded_texts = tokenizer.texts_to_sequences(text)
word_index = tokenizer.index_word
num_words = len(word_index) + 1
# encoded_texts = np.array(encoded_texts)

In [6]:
def windowize_data(data, n_prev):
    data = np.array(data)
    n_predictions = len(data) - n_prev
    y = data[n_prev:]
    indices = np.arange(n_prev) + np.arange(n_predictions)[:, None]
    x = data[indices]
    return x, y

In [44]:
len(encoded_texts)

TypeError: unhashable type: 'list'

In [8]:
n_prev = 20
X, y = windowize_data(encoded_texts[0], n_prev)
for text in encoded_texts[1:]:
    temp_X, temp_y = windowize_data(text,n_prev)
    X = np.concatenate((X, temp_X), axis=0)
    y = np.concatenate((y, temp_y), axis=0)

In [9]:
X = X[:25000,:]
y = y[:25000]

In [10]:
y = to_categorical(y, num_classes = num_words)

In [11]:
X.shape, y.shape

((25000, 20), (25000, 24775))

In [12]:
model = keras.Sequential()
model.add(Embedding(num_words, 128, input_length=n_prev))
model.add(LSTM(128, input_shape=(n_prev,1), return_sequences=True))
# model.add(Dropout(.2))
model.add(LSTM(128))
# model.add(Dropout(.2))
model.add(Dense(num_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [16]:
callbacks = [ModelCheckpoint(('../models/model.h5'), save_best_only=True, save_weights_only=False, monitor='val_accuracy')]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 128)           3171200   
_________________________________________________________________
lstm (LSTM)                  (None, 20, 128)           131584    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 24775)             3195975   
Total params: 6,630,343
Trainable params: 6,630,343
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(X, y, epochs=100, batch_size=32, callbacks=callbacks)

Epoch 1/100
782/782 [==============================] - 57s 73ms/step - loss: 6.6226 - accuracy: 0.0838
Epoch 2/100
782/782 [==============================] - 56s 72ms/step - loss: 6.3146 - accuracy: 0.0996
Epoch 3/100
782/782 [==============================] - 53s 67ms/step - loss: 6.0536 - accuracy: 0.1113
Epoch 4/100
782/782 [==============================] - 54s 70ms/step - loss: 5.8182 - accuracy: 0.1247
Epoch 5/100
782/782 [==============================] - 55s 70ms/step - loss: 5.6171 - accuracy: 0.1343
Epoch 6/100
782/782 [==============================] - 53s 68ms/step - loss: 5.4264 - accuracy: 0.1403
Epoch 7/100
782/782 [==============================] - 55s 71ms/step - loss: 5.2388 - accuracy: 0.1471
Epoch 8/100
782/782 [==============================] - 53s 68ms/step - loss: 5.0524 - accuracy: 0.1511
Epoch 9/100
782/782 [==============================] - 55s 71ms/step - loss: 4.8745 - accuracy: 0.1605
Epoch 10/100
782/782 [==============================] - 54s 69ms/step - l

Epoch 45/100
782/782 [==============================] - 53s 68ms/step - loss: 1.9671 - accuracy: 0.5736
Epoch 46/100
782/782 [==============================] - 55s 70ms/step - loss: 1.8192 - accuracy: 0.6022
Epoch 47/100
782/782 [==============================] - 52s 67ms/step - loss: 1.7590 - accuracy: 0.6138
Epoch 48/100
782/782 [==============================] - 54s 69ms/step - loss: 1.7169 - accuracy: 0.6250
Epoch 49/100
782/782 [==============================] - 54s 69ms/step - loss: 1.7215 - accuracy: 0.6235
Epoch 50/100
782/782 [==============================] - 53s 68ms/step - loss: 1.6995 - accuracy: 0.63300s - loss:
Epoch 51/100
782/782 [==============================] - 55s 70ms/step - loss: 1.6920 - accuracy: 0.6370
Epoch 52/100
782/782 [==============================] - 53s 68ms/step - loss: 1.6388 - accuracy: 0.6482
Epoch 53/100
782/782 [==============================] - 55s 70ms/step - loss: 1.5304 - accuracy: 0.6681
Epoch 54/100
782/782 [==============================] 

Epoch 89/100
782/782 [==============================] - 52s 67ms/step - loss: 0.5440 - accuracy: 0.8983
Epoch 90/100
782/782 [==============================] - 54s 68ms/step - loss: 0.5202 - accuracy: 0.9031
Epoch 91/100
782/782 [==============================] - 53s 68ms/step - loss: 0.4914 - accuracy: 0.9109
Epoch 92/100
782/782 [==============================] - 52s 67ms/step - loss: 0.4636 - accuracy: 0.9186
Epoch 93/100
782/782 [==============================] - 55s 70ms/step - loss: 0.4412 - accuracy: 0.9232
Epoch 94/100
782/782 [==============================] - 52s 67ms/step - loss: 0.4213 - accuracy: 0.9282
Epoch 95/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4021 - accuracy: 0.9319
Epoch 96/100
782/782 [==============================] - 52s 67ms/step - loss: 0.3796 - accuracy: 0.9366
Epoch 97/100
782/782 [==============================] - 54s 70ms/step - loss: 0.3562 - accuracy: 0.9436
Epoch 98/100
782/782 [==============================] - 52s 67ms

In [19]:
poetry_length = 10
def generate_poetry(seed_text, n_lines):
  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = tokenizer.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=20, padding='pre')

      y_pred = np.argmax(model.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in tokenizer.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' ' + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = ' '.join(text)
    print(text)

In [31]:
generate_poetry('I tried so hard, and got so far, but in the end it', 3)

was a hyena “the island in the water is far
until it had burst upon us but i could easily
in motion and sending forth to the bolt and went


In [25]:
model.save('../models/first_model')

In [40]:
seed_2 = "For arms there dangled from the upper portion of the carcass two tolerably long bottles, with the necks outward for hands."

In [41]:
generate_poetry(seed_2, 2)

and this vortex the whole surface of the household—of a
remembrance of aerial forms—of spiritual and meaning eyes—of sounds musical


In [37]:
data.text[25]

'AN EXTRAVAGANZA.     IT was a chilly November afternoon. I had just consummated an unusually hearty dinner, of which the dyspeptic truffe formed not the least important item, and was sitting alone in the dining-room, with my feet upon the fender, and at my elbow a small table which I had rolled up to the fire, and upon which were some apologies for dessert, with some miscellaneous bottles of wine, spirit and liqueur. In the morning I had been reading Glover’s “Leonidas,” Wilkie’s “Epigoniad,” Lamartine’s “Pilgrimage,” Barlow’s “Columbiad,” Tuckermann’s “Sicily,” and Griswold’s “Curiosities”; I am willing to confess, therefore, that I now felt a little stupid. I made effort to arouse myself by aid of frequent Lafitte, and, all failing, I betook myself to a stray newspaper in despair. Having carefully perused the column of “houses to let,” and the column of “dogs lost,” and then the two columns of “wives and apprentices runaway,” I attacked with great resolution the editorial matter, an

In [43]:
model.save('../models/whole_first_model')

INFO:tensorflow:Assets written to: ../models/whole_first_model/assets


INFO:tensorflow:Assets written to: ../models/whole_first_model/assets
